### Sample RHESSys workflow

**Note:** This workflow is underdevelopment and still needs documentation

In [1]:
from rhessys_wf import *

project_location = './test'
project_name = 'mytest'
gageid = '01589312'
start_date = '2008-01-01'
end_date = '2010-01-01'
rhessys_source_location = ''
publisher = 'RHESSysWorkflow'

!rm -rf test/
!mkdir test

In [2]:
import logging

# Get root logger (all other loggers will be derived from this logger's properties)
logger = logging.getLogger()

# assuming only a single handler has been setup (seems 
# to be default in notebook), set that handler to go to stdout.
logger.handlers[0].stream = sys.stdout

In [3]:
w = RHESSysWorkflow(project_location, project_name, gageid, start_date, end_date, rhessys_source_location, publisher)

Log file location: ./test/mytest/mytest.log


In [4]:
w.get_NHDStreamflowGageIdentifiersAndLocation(w.sub_project_folder,w.gageid)
w.get_CatchmentShapefileForNHDStreamflowGage(w.sub_project_folder)
w.get_BoundingboxFromStudyareaShapefile(w.sub_project_folder)
extent = w.get_Extent_from_RHESSysWorkflows_Metadata_File()

INFO:myApp:GetNHDStreamflowGageIdentifiersAndLocation.py -p ./test/mytest/mytest -g 01589312
INFO:myApp:GetCatchmentShapefileForNHDStreamflowGage.py --overwrite -p ./test/mytest/mytest
INFO:myApp:GetBoundingboxFromStudyareaShapefile.py -p ./test/mytest/mytest
INFO:myApp:./test/mytest/mytest/metadata.txt
INFO:myApp:
min x = -76.769782
min y = 39.273610
max x = -76.717498
max y = 39.326008



In [5]:
w.HydroTerre_RHESSys_ByExtent(extent, w.ht_start_date, w.ht_end_date, w.sub_project_folder)
zipfolder = w.sub_project_folder + '/RHESSys_ETV'
w.create_path(zipfolder)
zipfilepathname = w.sub_project_folder + '/RHESSys_ETV_Data.zip'
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)

checking to see if HydroTerre job is completed...
checking to see if HydroTerre job is completed...
--------Download Start-------------
Retrieving result from: http://hydroterre.psu.edu/HydroTerre_Rhessys_ByExtent/ja875f2ab65f34eee8e7f87f47cd9b099/scratch/RHESSys_ETV_Data.zip
--------Download End-------------
--------HUC12 Area-------------
412.045597076416 SQKM


In [6]:
output = w.get_USGSDEMForBoundingbox(w.sub_project_folder)
output = w.get_USGSNLCDForDEMExtent(w.sub_project_folder)
output = w.get_SSURGOFeaturesForBoundingbox(w.sub_project_folder)
output =w.GenerateSoilPropertyRastersFromSSURGO(w.sub_project_folder)
w.lai_fullpathname_with_ext = w.sub_project_folder + '/RHESSys_ETV/RHESSys_LAI/LAI_Month0.tif'
output = w.Register_LAI_Raster(w.sub_project_folder, w.lai_fullpathname_with_ext, w.publisher)
output = w.CreateGRASSLocationFromDEM(w.sub_project_folder, '"RHESSys model for Dead Run 5 watershed near Catonsville, MD"')

INFO:myApp:GetUSGSDEMForBoundingbox.py --overwrite -p ./test/mytest/mytest
INFO:myApp:GetUSGSNLCDForDEMExtent.py -p ./test/mytest/mytest
INFO:myApp:GetSSURGOFeaturesForBoundingbox.py -p ./test/mytest/mytest
INFO:myApp:GenerateSoilPropertyRastersFromSSURGO.py -p ./test/mytest/mytest
INFO:myApp:RegisterRaster.py -p ./test/mytest/mytest -t lai -r ./test/mytest/mytest/RHESSys_ETV/RHESSys_LAI/LAI_Month0.tif -b "RHESSysWorkflow" --force 
INFO:myApp:CreateGRASSLocationFromDEM.py -p ./test/mytest/mytest -d "RHESSys model for Dead Run 5 watershed near Catonsville, MD"


In [7]:
output = w.ImportRHESSysSource(w.sub_project_folder)
print 'Finished'

INFO:myApp:ImportRHESSysSource.py --overwrite -p ./test/mytest/mytest
Finished


In [13]:
output = w.climate_data_fullpathname = w.sub_project_folder + '/RHESSys_ETV/RHESSys_Climate'

output = w.ImportClimateData(w.sub_project_folder, w.climate_data_fullpathname)

output = w.station_data_fullpathname = w.sub_project_folder + '/RHESSys_ETV/RHESSys_Climate'

output = w.DelineateWatershed(w.sub_project_folder, w.dem_cell_threshold, w.areaEstimate)

output = w.GeneratePatchMap(w.sub_project_folder)

Using raster named 'dem' to calculate flow direction map
SECTION 1a (of 4): Initiating Memory.
SECTION 1b (of 4): Determining Offmap Flow.
   0%   4%   8%  12%  16%  20%  24%  28%  32%  36%  40%  44%  48%  52%  56%  60%  64%  68%  72%  76%  80%  84%  88%  92%  96% 100%
SECTION 2: A * Search.
   0%   2%   4%   6%   8%  10%  12%  14%  16%  18%  20%  22%  24%  26%  28%  30%  32%  34%  36%  38%  40%  42%  44%  46%  48%  50%  52%  54%  56%  58%  60%  62%  64%  66%  68%  70%  72%  74%  76%  78%  80%  82%  84%  86%  88%  90%  92%  94%  96%  98% 100%
SECTION 3: Accumulating Surface Flow with SFD.
   1%   3%

In [ ]:
output = w.GenerateSoilTextureMap(w.sub_project_folder)

output = w.ImportRasterMapIntoGRASS_LAI(w.sub_project_folder)

output = w.ImportRasterMapIntoGRASS_LANDCOVER(w.sub_project_folder)

In [ ]:
output = w.RegisterLandcoverReclassRules(w.sub_project_folder)
output = w.GenerateLandcoverMaps(w.sub_project_folder)
output = w.GenerateWorldTemplate(w.sub_project_folder)
output = w.CreateWorldfile(w.sub_project_folder)
output = w.CreateFlowtable(w.sub_project_folder)
output = w.RunLAIRead(w.sub_project_folder)
output = w.RunCmd(w.sub_project_folder, 3)            

In [ ]:
output = w.RunModel(w.sub_project_folder)

In [ ]:
obs_data = '/projects/start_from_scratch/01589312/DR5_discharge_WY2008-2012.txt'
output = w.RHESSysPlot(w.sub_project_folder, obs_data)